In [ ]:
import sys
sys.path.append('..')
from theano import function, config, shared, sandbox
import theano.tensor as T
import numpy
import time
import theano_lstm
import numpy as np

In [ ]:
vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], T.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in xrange(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

In [ ]:
num_examples = 128
input_seq = np.random.normal(size=(2200, 300, num_examples)).astype('float32')
seq_lengths = [np.random.randint(low=500,high=2200) for x in range(num_examples)]
answers = np.zeros(shape=(50, num_examples)).astype('float32')
answers[10, :] = 1

In [ ]:
%%time
reload(theano_lstm)
config.mode="FAST_COMPILE"
hidden_size = 256
network = theano_lstm.lstm_rnn(300,
                  [(hidden_size, hidden_size),
                   (hidden_size, hidden_size)],
                  50, log_dir='test_log')
network.initialize_training_adadelta()

In [ ]:
network_opt.initialize_network_weights()

In [ ]:
start_time = time.time()
_=network_opt.adadelta_step(input_seq, seq_lengths, answers)
duration = time.time() - start_time
print duration

In [ ]:
start_time = time.time()
_=network.adadelta_step(input_seq, seq_lengths, answers)
duration = time.time() - start_time
print duration

In [ ]:
%%time
reload(theano_lstm)
config.mode="FAST_RUN"
hidden_size = 256
network_opt = theano_lstm.lstm_rnn(300,
                  [(hidden_size, hidden_size),
                   (hidden_size, hidden_size)],
                  50, log_dir='test_log')
network_opt.initialize_training_adadelta()

In [ ]:
%%time
_=network_opt.adadelta_step(input_seq, seq_lengths, answers)

In [ ]:
%%time
network_opt.write_parameters()

In [ ]:
range(10, 100, 10)

In [ ]:
duration = []
for batch_size in range(10, 500, 25):
    num_examples = batch_size
    input_seq = np.random.normal(size=(2200, 300, num_examples)).astype('float32')
    seq_lengths = [np.random.randint(low=500,high=2200) for x in range(num_examples)]
    answers = np.zeros(shape=(50, num_examples)).astype('float32')
    answers[10, :] = 1
    
    start_time = time.time()
    _=network.adadelta_step(input_seq, seq_lengths, answers)
    duration.append(time.time() - start_time)

In [ ]:
range(10,500,25)[:len(duration)]

In [ ]:
duration

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(range(10,500,25)[:len(duration)], duration);

In [ ]:
config.allow_gc